In [ ]:

using Pkg
Pkg.add("Plots")
Pkg.add("Random")
Pkg.add("GLM")
Pkg.add("DataFrames")
Pkg.add("LaTeXStrings")
Pkg.add("Statistics") 
Pkg.add("StatsBase")
Pkg.add("CSV")
Pkg.add("CategoricalArrays")

In [1]:
#2 Overfitting 

using Plots
using Random
using GLM
using DataFrames
using LaTeXStrings
using Statistics
using StatsBase
using CSV

function datasplitting(data,proportion)

    sampling = sample([1:size(data,1);],Int(proportion*size(data,1)),replace=false) #sample indexes
    sampling = sort(sampling) # sort index
    train = data[sampling,:] # subset of dataframe with sampled indexes -> train set
    remaining = [i for i in 1:size(data,1) if isempty(searchsorted(sampling, i))] #remaining indexes of the data frame
    test = data[remaining, :] #s ubset of dataframe with remaining indexes -> test set

    return train, test
end  


Random.seed!(1234)
X=rand(Float64,1000);
X=sort(X)
e=randn(1000);
Y=exp.(4X)+e;    
data=DataFrame([Y],["Y"])
covariates=names(data)[2:end]

datasets = Dict()

R_cuadrados = DataFrame(features = Int[],R_squared = Float64[],adjusted_R_squared = Float64[],out_of_sample_R_squared = Float64[])

for features in [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
    df_copy = deepcopy(data)
    for i in 1:features
        df_copy[!, Symbol("X", i)] = X .^ i
    end
    datasets[features] = df_copy
    covariates_aux=names(datasets[features])[2:end]

    model=lm(term(:Y)~sum(term.(covariates_aux)), datasets[features]);
    mse=mean((Y-predict(model,select(datasets[features],covariates_aux))).^2)
    R_sq=1-mse/mean((Y.-mean(Y)).^2)
    adj_mse=1000/(1000-features)*mse
    adjR_sq=1-adj_mse/mean((Y.-mean(Y)).^2)
    adjR_sq


    train,test=datasplitting(datasets[features],0.75)
    model_training=lm(term(:Y)~sum(term.(covariates_aux)),train);
    Y_test = test[:,"Y"];
    mse_test=mean((Y_test -predict(model_training,select(test,covariates_aux))).^2)
    R_sq_test = 1 - mse_test/mean((Y_test.-mean(Y_test)).^2)

    push!(R_cuadrados,(features,R_sq,adjR_sq,R_sq_test))
end

CSV.write("R squares.csv",R_cuadrados)

p1 = plot(R_cuadrados.features, R_cuadrados.R_squared,
    xlabel="Number of features", ylabel="R-squared",
    title="R-squared vs Number of features",
    lw=2, marker=:circle, ylims=(0.79,1))

p2 = plot(R_cuadrados.features, R_cuadrados.adjusted_R_squared,
    xlabel="Number of features", ylabel="Adjusted R-squared",
    title="Adjusted R-squared vs Number of features",
    lw=2, marker=:circle, ylims=(0.79,1))
p3 = plot(R_cuadrados.features, R_cuadrados.out_of_sample_R_squared,
    xlabel="Number of features", ylabel="Out-of-sample R-squared",
    title="Out-of-sample R-squared vs Number of features",
    lw=2, marker=:circle, ylims=(0.79,1))

savefig(p1, "R-squared.pdf")
savefig(p2, "Adjusted R-squared.pdf")
savefig(p3, "Out-of-sample R-squared.pdf")
#Para el caso del R-squared  conforme se aumenta el número de features, este aumenta. Tiene sentido ya que a mayor numero de variables, el modelo captura mejor la variación de la variable dependiente.
#El Adjusted R-squared cae conforme aumenta el número de features. Esto es asi porque este indicador castiga la cantidad de regresores que se usa en el modelo.   
#El Out-of-sample R-squared aumenta al principio, pero luego cae un poco. Sin embargo con el mayor numero de features aumenta más. Parece que el modelo predice bien para datos fuera de la muestra.

"c:\\Users\\User\\Desktop\\UP y educacion\\Semestre 2025-II-Ciclo 10-UP\\Inferencia Causal y ML\\Trabajo\\Scripts\\Out-of-sample R-squared.pdf"